### Görev 1: Parametre Verimliliğini Kanıtlama (The "Efficiency" Challenge)

Aşağıdaki kod bloğunda eksik bırakılan yerleri tamamlayın. Amacımız, LoRA uygulandığında eğitilebilir parametre sayısının ne kadar düştüğünü (veya ne kadar az parametre ile eğitim yapıldığını) matematiksel olarak görmektir.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType

# 1. Base Modeli Yükle (Hafif olması için GPT-2 kullanıyoruz)
model_name = "gpt2" 
print(f"Model yükleniyor: {model_name}...")
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def print_trainable_parameters(model):
    """
    Modeldeki eğitilebilir parametre sayısını ve tüm parametrelere oranını yazdırır.
    """
    trainable_params = 0
    all_param = 0
    
    # GÖREV: Modelin parametreleri üzerinde döngü kurarak sayım yapın
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
            
    print(f"Eğitilebilir Parametre: {trainable_params} || Tümü: {all_param} || Oran: %{100 * trainable_params / all_param:.4f}")

print("\n--- Base Model Durumu ---")
print_trainable_parameters(model)

# 2. LoRA Konfigürasyonu Oluştur
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    inference_mode=False, 
    r=8,            # Rank değeri
    lora_alpha=32,  # Alpha ölçekleme değeri
    lora_dropout=0.1
)

# 3. Modeli PEFT Model'e Dönüştür
peft_model = get_peft_model(model, peft_config)

print("\n--- LoRA Entegre Edilmiş Model Durumu ---")
# Fonksiyonu tekrar çağırarak farkı görün
print_trainable_parameters(peft_model)

### Görev 2: LoRA Katmanlarını Gözlemleme (The "Architecture" Challenge)

LoRA, modelin mevcut katmanlarına (genellikle Attention bloklarına) küçük matrisler ekler. Aşağıdaki kodu çalıştırarak `lora_A` ve `lora_B` katmanlarının modelin neresine eklendiğini çıktı üzerinde tespit edin.

In [ ]:
# Modelin mimarisini yazdır
print("Model Mimarisi ve LoRA Katmanları:\n")
print(peft_model)

# GÖREV İPUCU: Çıktı içerisinde "lora_A" ve "lora_B" ifadelerini arayın. 
# Hangi katmanların altında (Linear, Conv1D vb.) göründüğüne dikkat edin.